In [7]:
import scqubits as scq
import qutip as qt
import numpy as np

def compute_chi_exact(qubit_type: str,
                      w_res_GHz: float,
                      g_MHz: float,
                      EC: float,
                      EJ: float,
                      EL: float = None,
                      ncut: int = 41,
                      q_dim: int = 10,
                      res_dim: int = 20):
    """
    Compute exact dispersive shift chi for a qubit (Transmon or Fluxonium)
    coupled to a resonator, using diagonalization.

    Args:
        qubit_type : "transmon" or "fluxonium"
        w_res_GHz  : resonator frequency (GHz)
        g_MHz      : coupling (MHz)
        EC, EJ     : qubit parameters (GHz)
        EL         : inductive energy for fluxonium (GHz), required if qubit_type="fluxonium"
        ncut       : charge cutoff (qubit basis size)
        q_dim      : truncated dimension for qubit Hilbert space
        res_dim    : resonator Hilbert space dimension

    Returns:
        chi_exact_GHz : dispersive shift χ (GHz)
    """
    g_GHz = g_MHz / 1e3


    # --- Build qubit
    if qubit_type.lower() == "transmon":
        qubit = scq.Transmon(EJ=EJ, EC=EC, ng=0, ncut=ncut, truncated_dim=q_dim)
    elif qubit_type.lower() == "fluxonium":
        if EL is None:
            raise ValueError("Fluxonium requires EL.")
        qubit = scq.Fluxonium(EJ=EJ, EC=EC, EL=EL, flux=0.0, cutoff=ncut, truncated_dim=q_dim)
    else:
        raise ValueError("qubit_type must be 'transmon' or 'fluxonium'.")

    # --- Build resonator
    resonator = scq.Oscillator(E_osc=w_res_GHz, truncated_dim=res_dim)

    # --- Hilbert space
    hs = scq.HilbertSpace([qubit, resonator])
    hs.add_interaction(
        expr="g * n * a - g * n * adag",
        op1=("n", qubit.n_operator),
        op2=("adag", resonator.creation_operator),
        op3=("a", resonator.annihilation_operator),
        add_hc=False,
        const=dict(g=1j * g_GHz)
    )
    hs.generate_lookup()

    # --- Exact chi from dressed energies
    evals = hs["evals"][0]
    diag_h = 2 * np.pi * qt.Qobj(np.diag(evals), dims=[hs.subsystem_dims]*2)
    diag_trunc = qt.Qobj(diag_h[:10, :10])
    evalues_GHz = (diag_trunc.eigenenergies() - diag_trunc.eigenenergies()[0]) / (2*np.pi)

    e_11 = evalues_GHz[hs.dressed_index((1,1))]
    e_10 = evalues_GHz[hs.dressed_index((1,0))]
    e_01 = evalues_GHz[hs.dressed_index((0,1))]
    e_00 = evalues_GHz[hs.dressed_index((0,0))]

    chi_exact_GHz = e_11 - e_10 - e_01 + e_00
    return chi_exact_GHz


In [46]:
# Transmon 
chi_t = compute_chi_exact("transmon", w_res_GHz=7.3, g_MHz=150.0, EC=0.167, EJ=16)
print("Transmon χ =", chi_t*1e3, "MHz")

# Fluxonium 
chi_f = compute_chi_exact("fluxonium", w_res_GHz=7.6, g_MHz=75/2, EC=.88, EJ=5, EL=1.2)
print("Fluxonium χ =", chi_f*1e3, "MHz")


Transmon χ = -2.4252128486086377 MHz
Fluxonium χ = -0.298299844787131 MHz
